In [1]:
%reload_ext autoreload
%autoreload 2


In [2]:
from indexing import Indexing

In [3]:
from vexa import VexaAPI
vexa = VexaAPI(token='3ae04e20124d40babc5107e658c666b6')
await vexa.get_user_info()

User information retrieved successfully.


{'id': 'ef7c085b-fdb5-4c94-b7b6-a61a3d04c210',
 'email': '2280905@gmail.com',
 'is_allowed_send_init_message': True,
 'username': 'Dmitriy Grankin',
 'first_name': 'Dmitriy',
 'last_name': 'Grankin',
 'image': 'https://lh3.googleusercontent.com/a/ACg8ocLu4ZPfzj9KFNVqKL3NTF4SXUYvJ4mwoXLuoQ0BZzjw7OBIRA=s96-c',
 'created_timestamp': '2024-05-28T11:16:50.894711Z',
 'updated_timestamp': '2024-10-18T08:47:02.519781Z'}

In [6]:
from indexing import check_item_exists, process_meeting_data, save_meeting_data_to_db

In [11]:

self = Indexing(token=vexa.token)

In [12]:
num_meetings = 100

In [14]:
import asyncio

In [16]:
from pydantic_models import MeetingExtraction, EntityExtraction, SummaryIndexesRefs

In [18]:
import pandas as pd

In [19]:
async def process_meeting_data(formatted_input, df):
    extraction_tasks = [
        MeetingExtraction.extract(formatted_input),
        EntityExtraction.extract(formatted_input)
    ]
    discussion_points_df, topics_df = await asyncio.gather(*extraction_tasks)
    
    discussion_points_df['model'] = 'MeetingExtraction'
    topics_df['model'] = 'EntityExtraction'
    
    # Rename columns to match the new schema
    discussion_points_df = discussion_points_df.rename(columns={'item': 'topic_name', 'type': 'topic_type'})
    topics_df = topics_df.rename(columns={'entity': 'topic_name', 'type': 'topic_type'})
    
    # Combine the dataframes
    summary_df = pd.concat([discussion_points_df, topics_df]).reset_index(drop=True)
    
    summary_refs = await SummaryIndexesRefs.extract(summary_df, formatted_input)

    # Create a new dataframe for the references
    ref_df = pd.DataFrame([(ref['summary_index'], r['s'], r['e']) 
                           for ref in summary_refs 
                           for r in ref['references']],
                          columns=['summary_index', 'start', 'end'])

    # Merge the ref_df with summary_df
    entities_with_refs = summary_df.reset_index().rename(columns={'index': 'summary_index'})
    entities_with_refs = entities_with_refs.merge(ref_df, on='summary_index', how='left')

    # Function to extract text from df based on start and end indices, including speaker
    def get_text_range_with_speaker(row):
        text_range = df.loc[row['start']:row['end']]
        return ' | '.join(f"{speaker}: {content}" for speaker, content in zip(text_range['speaker'], text_range['content']))

    # Apply the function to get the referenced text with speakers
    entities_with_refs['referenced_text'] = entities_with_refs.apply(get_text_range_with_speaker, axis=1)

    # Group by summary_index to combine multiple references
 #   try:
    final_df = entities_with_refs.groupby('summary_index').agg({
        'topic_name': 'first',
        'topic_type': 'first',
        'summary': 'first',
        'details': 'first',
        'speaker': 'first',
        'referenced_text': ' | '.join,
        'model': 'first'
    }).reset_index()

    return final_df
    # except Exception as e:
    #     print(f"Error processing meeting data: {e}")
    #     return pd.DataFrame()

In [20]:
meetings = await self.vexa.get_meetings()
meetings = meetings[-num_meetings:]  # Process last N meetings

for meeting in meetings:
    meeting_id = meeting['id']
    # try:
    if not await check_item_exists(meeting_id):
        transcription = await self.vexa.get_transcription(meeting_session_id=meeting_id, use_index=True)
        if transcription:   
            df, formatted_input, start_datetime, speakers, transcript = transcription
            final_df = await asyncio.wait_for(
                process_meeting_data(formatted_input, df),
                timeout=60
            )
            await save_meeting_data_to_db(final_df, meeting_id, transcript, start_datetime)

Request URL: http://127.0.0.1:8001/api/v1/calls/all
Request Params: {'token': '3ae04e20124d40babc5107e658c666b6'}


KeyError: "Column(s) ['details', 'speaker', 'summary', 'topic_name', 'topic_type'] do not exist"

In [ ]:

await indexing.index_meetings(user_id='', num_meetings=200)